In [1]:
### Enter your path for downloaded directory NL2STL
#home_path_nl2stl = 'YOUR-OWN-PATH/NL2STL'
#%cd 'YOUR-OWN-PATH/NL2STL'

home_path_nl2stl = '/content/drive/MyDrive/Colab Notebooks/Code/NLP_robotics/LTL_dataset/github' ### Enter your path for downloaded directory NL2STL
%cd '/content/drive/MyDrive/Colab Notebooks/Code/NLP_robotics/LTL_dataset/github'

/content/drive/MyDrive/Colab Notebooks/Code/NLP_robotics/LTL_dataset/github


In [ ]:
!pip install transformers
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 8.5 MB/s 
     |████████████████████████████████| 182 kB 61.3 MB/s 
     |████████████████████████████████| 7.6 MB 46.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.9 MB/s 


In [ ]:
import os
import json
import time
from argparse import ArgumentParser
import tqdm
import torch
import torch.nn as nn

from torch.utils.data import DataLoader

from transformers import T5Tokenizer, AdamW, get_linear_schedule_with_warmup
from seq2seq import LTL2Eng
from data import LTLDataset, generate_vocabs, LTL2EngT5Dataset, UnlabeledLTLDataset
from config import Config
from util import collect_ltl_vocabs

config = Config.from_json_file('eng2ltl_t5_load_data.json')
input_dir = os.path.join(home_path_nl2stl, 'eng2ltl_para_gen_5_11_28_word_infix')

torch.cuda.set_device(0)
model_name = config.bert_model_name
tokenizer = T5Tokenizer.from_pretrained(model_name,cache_dir=config.bert_cache_dir)

ltl_vocabs = collect_ltl_vocabs([home_path_nl2stl+config.train_file,home_path_nl2stl+config.dev_file,home_path_nl2stl+config.test_file])
model = LTL2Eng(config,ltl_vocabs,tokenizer)
model.load_state_dict(torch.load(input_dir+'/model_state.pt'))
model.cuda(device=0)
model.eval()

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

LTL2Eng(
  (ltl2eng): T5ForConditionalGeneration(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=768, out_features=2048, bias=False)
                (wi_1): Linear(in_

In [6]:
import pandas as pd
# Here you can define the imperative natural sentences to express orders, like the following example
# Here we represent atomic propositions with ( prop_* )
'''
test_sentence = ['If ( prop_4 ) happens and implies ( prop_2 ) and this scenario continues to hold until at some point during the 243 to 582 time units ( prop_3 ) is detected , then the scenario is equivalent to ( prop_1 ) .',
                 'If it is not the case that ( prop_3 ) is detected for each time instant in the coming 164 to 612 time units , or else ( prop_1 ) happens , then ( prop_2 ) .',
                 'If at some point ( prop_3 ) and ( prop_2 ) , and is equivalent to ( prop_4 ) , and this scenario will hold until at some other point ( prop_1 ) is detected .',  
                 '( prop_1 ) happens implies ( prop_2 ) happens',               
                 
                 ]
'''

'''
# Another way is to load the excel files, there are some example files in the dataset dir, also here we use one example file
file_name = 'example_excel_nl.xlsx'
df = pd.read_excel(file_name)
test_sentence = [df['paraphrased_logic_sentence'][i] for i in range(len(df))]
'''


# Another way is to load the txt file
my_file = open("example_excel_nl.txt", "r")
# reading the file
data_test_sentence = my_file.read()

# replacing end splitting the text 
# when newline ('\n') is seen.
test_sentence = data_test_sentence.split("\n")
my_file.close()


In [ ]:
import os
home_path_output = home_path_nl2stl + '/application_test/'
if not os.path.exists(home_path_output):
  os.mkdir(home_path_output)

dataset_total = [];
with open(home_path_output+"test1_word_infix.jsonl", "w") as outfile:
    for i in range(len(test_sentence)):
      dataset_item = {};
      dataset_item['id'] = i
      dataset_item['logic_ltl'] = ''
      dataset_item['logic_sentence'] = test_sentence[i].split(' ')
      outfile.write(json.dumps(dataset_item)+'\n')
outfile.close()

test_set = LTL2EngT5Dataset(home_path_output+"test1_word_infix.jsonl",tokenizer,config,model.ltl2eng.config,ltl_vocabs=ltl_vocabs)
data_batch =  DataLoader(test_set, batch_size=1,
                        shuffle=False, collate_fn=test_set.collate_fn)

68it [00:00, 362.50it/s]

Loaded 68 instances from /content/drive/MyDrive/Colab Notebooks/Code/NLP_robotics/LTL_dataset/github/application_test/test1_word_infix.jsonl


In [ ]:
pred_test = []
for i, batch in enumerate(data_batch):
  if i <len(test_sentence):
    outputs = model.predict_eng2ltl(batch, max_length=config.max_generate_length)
    pred_ltls = tokenizer.batch_decode(outputs['output_idxs'], skip_special_tokens=True)
    print(test_sentence[i])
    print(pred_ltls[0])
    print('\n')
    pred_test.append(pred_ltls[0])
  else:
    break

For all time instants in the future ( prop_2 ) should be detected , or ( prop_1 ) happens in one time , or else ( prop_3 ) happens in one time .
( ( globally ( prop_2 or prop_1 ) or prop_3 )


If ( prop_3 ) then ( prop_4 ) , then ( prop_2 ) , or else ( prop_1 ) .
( ( ( prop_3 imply prop_4 ) imply prop_2 ) or prop_1 )


If at some point ( prop_4 ) or ( prop_2 ) then the scenario is equivalent to ( prop_1 ) , it should be followed by ( prop_3 ) .
( ( ( prop_4 or prop_2 ) equal prop_1 ) imply prop_3 )


If ( prop_1 ) and ( prop_4 ) are equivalent to ( prop_3 ) , and also ( prop_2 ) .
( ( ( prop_1 and prop_4 ) equal prop_3 ) and prop_2 )


If ( prop_3 ) does not happen and also ( prop_1 ) happens , then ( prop_2 ) .
( ( negation prop_3 and prop_1 ) imply prop_2 )


If ( prop_1 ) is equivalent to ( prop_2 ) or else ( prop_3 ) and also ( prop_4 ) .
( ( ( prop_1 equal prop_2 ) or prop_3 ) and prop_4 )


It is required that at a certain point ( prop_2 ) and ( prop_3 ) , or else ( prop_1 ) .
( 